In [1]:
import numpy as np

In [2]:
sc #spark context

In [3]:
spark 

In [4]:
sqlContext #spark data table context

In [5]:
a = range(10)

In [6]:
a

range(0, 10)

In [7]:
rdd = sc.parallelize(a) #lazy send to spark
rdd

PythonRDD[1] at RDD at PythonRDD.scala:48

In [8]:
rdd.first()

0

In [9]:
rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
rdd =rdd.map(lambda x: x*10) #lazy! it return another rdd
rdd

PythonRDD[3] at RDD at PythonRDD.scala:48

In [11]:
rdd.reduce(lambda x,y:x+y) # not lazy (action)

450

In [12]:
rdd = rdd.filter(lambda x: x>30)
rdd

PythonRDD[5] at RDD at PythonRDD.scala:48

In [13]:
rdd.collect()

[40, 50, 60, 70, 80, 90]

## Exercises
1. Get an RDD with numbers 2 to 10
2. Get all elements that are bigger than 5
3. Get the product of the elements of the result of 2

## Input

In [17]:
allcsv = sc.textFile("data/*.txt")

In [18]:
allcsv.first()

'Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context'

In [19]:
allcsv.map(lambda x: x.split(",")).first()

['Crime ID',
 'Month',
 'Reported by',
 'Falls within',
 'Longitude',
 'Latitude',
 'Location',
 'LSOA code',
 'LSOA name',
 'Crime type',
 'Last outcome category',
 'Context']

## Tuples and ReduceByKey
First element of a tuple is considered as key

In [20]:
data =  [('a',7), ('a', 2), ('b', 2), ('b',4), ('c',1), ('c',2), ('c',3), ('c',4)]

In [21]:
RDD = sc.parallelize(data) #send data to spark

In [22]:
RDD.collect()

[('a', 7),
 ('a', 2),
 ('b', 2),
 ('b', 4),
 ('c', 1),
 ('c', 2),
 ('c', 3),
 ('c', 4)]

In [23]:
RDD.reduceByKey(lambda x,y: x+y).collect()

[('a', 9), ('c', 10), ('b', 6)]

In [24]:
data = [['Alexandra','31','F','Python'],['Carla','25','F','C'],['Max','18','M','Scala'],['Tom','34','M','C'],['Philip','28','M','Python'],['Lucy','25','F','Scala'],['Al','18','M','Scala'],['Grace','34','F','Python']]

In [25]:
RDD = sc.parallelize(data)

In [26]:
RDD.reduceByKey?

In [27]:
RDD.map(lambda t: (t[2],1)).reduceByKey(lambda x,y: x+y).collect()

[('F', 4), ('M', 4)]

In [28]:
RDD.map(lambda t: (t[3],int(t[1]))).first()

('Python', 31)

In [29]:
RDD.map(lambda t: (t[3],int(t[1]))).reduceByKey(lambda x,y:x+y).collect()

[('C', 59), ('Scala', 61), ('Python', 93)]

In [30]:
RDD.map(lambda t: (t[3],(int(t[1]),1))).collect()

[('Python', (31, 1)),
 ('C', (25, 1)),
 ('Scala', (18, 1)),
 ('C', (34, 1)),
 ('Python', (28, 1)),
 ('Scala', (25, 1)),
 ('Scala', (18, 1)),
 ('Python', (34, 1))]

In [31]:
RDD.map(lambda t: (t[3],(int(t[1]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).collect()

[('C', (59, 2)), ('Scala', (61, 3)), ('Python', (93, 3))]

In [32]:
RDD.map(lambda t: (t[3],(int(t[1]),1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x:(x[0],x[1][0]/x[1][1])).collect()

[('C', 29.5), ('Scala', 20.333333333333332), ('Python', 31.0)]

## Exercise
1. Compute the average age by gender (the key is the first element in the tuple)
2. Compute the preferred language by gender (use a tuple as a key)